In [15]:
import pandas as pd
from pandarallel import pandarallel
pandarallel.initialize(progress_bar=True)
from lib.mutation_util import get_tx_datetime
from apate.apate import ApateGraphFeatures

%load_ext autoreload
%autoreload

INFO: Pandarallel will run on 6 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [16]:
df_ori = pd.read_csv('../adyen-dataset.csv')
df_graph_features = df_ori.copy()[
    ["psp_reference", "merchant", "card_number","has_fraudulent_dispute"]]
df_graph_features["tx_datetime"] = df_ori.parallel_apply(get_tx_datetime, axis=1)  # for each row of df
df_graph_features["tx_datetime"] = pd.to_datetime(df_graph_features["tx_datetime"])
df_graph_features.head()

,psp_reference,merchant,card_number,has_fraudulent_dispute,tx_datetime
0,79622457320,Merchant A,XldVNN9TQgIA2RE9FTC2TA,False,2021-07-25 13:51:00
1,41293916020,Merchant C,zgRoL8pxX5LJI-SEPUTMvg,False,2021-12-11 15:20:00
2,31382555255,Merchant E,T3BF1uOOqgcwaVRb_ujPQg,False,2021-10-20 01:56:00
3,13944051314,Merchant D,BMf1zFkJ1xvPMlOxC7RWTw,False,2021-08-30 12:50:00
4,62343575411,Merchant E,jfxS2TIe0gR9vpOUdMKrBw,False,2021-11-14 19:26:00


## Adapt features to apate features

In [17]:
# ['Global_Date', 'Local_Date', 'CardID', 'MerchantID', 'Target']

In [18]:
df_graph_features["Global_Date"] = df_graph_features["tx_datetime"]
df_graph_features["Local_Date"] = df_graph_features["tx_datetime"]
df_graph_features["CardID"] = df_graph_features["card_number"]
df_graph_features["MerchantID"] = df_graph_features["merchant"]
df_graph_features["Target"] = df_graph_features["has_fraudulent_dispute"]
df_graph_features = df_graph_features[['psp_reference','Global_Date', 'Local_Date', 'CardID', 'MerchantID', 'Target']]

In [19]:
df_graph_features_learning = df_graph_features.loc[df_graph_features["Global_Date"].dt.month <= 6].copy()
df_graph_features_training_test = df_graph_features.loc[df_graph_features["Global_Date"].dt.month > 6].copy()
df_graph_features_learning.head()

,psp_reference,Global_Date,Local_Date,CardID,MerchantID,Target
5,25784401111,2021-06-11 13:21:00,2021-06-11 13:21:00,cjyrXCInVMph0CUUu9JSRg,Merchant B,False
8,58425890431,2021-01-31 07:04:00,2021-01-31 07:04:00,ozjVWKojjOrnCTXwe9_aRA,Merchant E,False
15,48607651692,2021-05-13 18:13:00,2021-05-13 18:13:00,-YSipQyucEelyic94vuHWQ,Merchant D,False
16,87548570452,2021-02-15 19:34:00,2021-02-15 19:34:00,EV8LQlDv30u4jmJzflO9tQ,Merchant A,False
17,86767652804,2021-04-12 11:24:00,2021-04-12 11:24:00,s91LpLlt4HKPS4yuD77sJw,Merchant B,False


## Generate Graph Features

In [20]:
graph_features = ApateGraphFeatures(df_graph_features_learning)

In [21]:
graph_features.add_graph_features(df_graph_features_training_test)

In [22]:
df_graph_features_training_test.head()

,psp_reference,Global_Date,Local_Date,CardID,MerchantID,Target,CHScore,CHScore_ST,CHScore_MT,CHScore_LT,MerScore,MerScore_ST,MerScore_MT,MerScore_LT,TrxScore,TrxScore_ST,TrxScore_MT,TrxScore_LT
0,79622457320,2021-07-25 13:51:00,2021-07-25 13:51:00,XldVNN9TQgIA2RE9FTC2TA,Merchant A,False,0.000004,0.000026,0.000004,0.000004,0.092327,0.086719,0.092361,0.092327,0.000009,0.000019,0.000009,0.000009
1,41293916020,2021-12-11 15:20:00,2021-12-11 15:20:00,zgRoL8pxX5LJI-SEPUTMvg,Merchant C,False,0.000008,0.000008,0.000008,0.000008,0.022136,0.029185,0.022120,0.022136,0.000010,0.000012,0.000010,0.000010
2,31382555255,2021-10-20 01:56:00,2021-10-20 01:56:00,T3BF1uOOqgcwaVRb_ujPQg,Merchant E,False,0.000008,0.000008,0.000008,0.000008,0.023406,0.016354,0.023352,0.023406,0.000011,0.000008,0.000011,0.000011
3,13944051314,2021-08-30 12:50:00,2021-08-30 12:50:00,BMf1zFkJ1xvPMlOxC7RWTw,Merchant D,False,0.000008,0.000008,0.000008,0.000008,0.045498,0.045892,0.045526,0.045498,0.000010,0.000010,0.000010,0.000010
4,62343575411,2021-11-14 19:26:00,2021-11-14 19:26:00,jfxS2TIe0gR9vpOUdMKrBw,Merchant E,False,0.000008,0.000008,0.000008,0.000008,0.023406,0.016354,0.023352,0.023406,0.000011,0.000008,0.000011,0.000011


In [23]:
df_graph_features_training_test.to_csv("graph_features_july_december.csv", index=False)